In [ ]:
import pandas as pd

df = pd.read_csv("Data_train_demo.csv")

df.head()

,type,group,education,meal,preparation course,score-1,score-2,score-3
0,wild,group B,some high school,standard,completed,63,67,67
1,domestic,group C,bachelor's degree,standard,none,67,69,75
2,domestic,group C,some college,standard,completed,69,90,88
3,domestic,group B,some high school,standard,none,62,64,66
4,wild,group E,some college,reduced,none,93,90,83


In [ ]:
df.shape

(700, 8)

In [ ]:
df.describe()

,score-1,score-2,score-3
count,700.000000,700.000000,700.000000
mean,66.028571,69.038571,67.828571
std,15.140138,14.589664,15.152323
min,0.000000,17.000000,10.000000
25%,56.000000,59.000000,57.000000
50%,66.000000,70.000000,68.000000
75%,77.000000,79.000000,78.000000
max,100.000000,100.000000,100.000000


In [ ]:
df.describe(include='object')

,type,group,education,meal,preparation course
count,700,680,700,700,700
unique,2,5,6,2,2
top,domestic,group C,associate's degree,standard,none
freq,357,207,154,444,442


In [ ]:
df['group'].fillna('Unknown', inplace=True)

In [ ]:
df['Pass'] = (df['score-1'] > 50) & (df['score-2'] > 50) & (df['score-3'] > 50)

df['Pass'] = df['Pass'].astype(int)

df.Pass

0      1
1      1
2      1
3      1
4      1
      ..
695    1
696    0
697    1
698    0
699    1
Name: Pass, Length: 700, dtype: int64

In [ ]:
len(df[df.Pass == 0])

145

In [ ]:
df[df.type == 'domestic']['Pass'].mean() - df[df.type == 'wild']['Pass'].mean()

0.022580460755730947

In [ ]:
df['score-1'].median()

66.0

Сколько кошек среди не прошедших полосу препятствий имели инструктора с уровнем образования "high school"?

In [ ]:
len(df[df.Pass == 0][df.education == 'high school'])

<ipython-input-34-e572c98a271e>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df[df.Pass == 0][df.education == 'high school'])


35

In [ ]:
df.columns

Index(['type', 'group', 'education', 'meal', 'preparation course', 'score-1',
       'score-2', 'score-3', 'Pass'],
      dtype='object')

In [ ]:
df_new = pd.get_dummies(df, columns=['type','group','education','meal','preparation course'], drop_first=True)

In [ ]:
df_new.shape

(700, 16)

In [ ]:
df_new.head()

,score-1,score-2,score-3,Pass,type_wild,group_group B,group_group C,group_group D,group_group E,education_bachelor's degree,education_high school,education_master's degree,education_some college,education_some high school,meal_standard,preparation course_none
0,63,67,67,1,1,1,0,0,0,0,0,0,0,1,1,0
1,67,69,75,1,0,0,1,0,0,1,0,0,0,0,1,1
2,69,90,88,1,0,0,1,0,0,0,0,0,1,0,1,0
3,62,64,66,1,0,1,0,0,0,0,0,0,0,1,1,1
4,93,90,83,1,1,0,0,0,1,0,0,0,1,0,0,1


In [ ]:
X = df_new.drop('Pass', axis=1) # матрица объект-признак
y = df_new['Pass'] # целевая переменная

Обучите решающее дерево (DecisionTreeClassifier из библиотеки sklearn.tree) глубины 5 с энтропийным критерием информативности на закодированных в пункте а) тренировочных данных по кросс-валидации с тремя фолдами, метрика качества - roc-auc.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

tree = DecisionTreeClassifier(max_depth = 5, criterion='entropy')

cross_val_score(tree, X, y, cv=3, scoring='roc_auc').mean()

0.9965986394557823

In [ ]:
?DecisionTreeClassifier

Подберите глубину решающего дерева (max_depth), перебирая глубину от 2 до 20 с шагом 1 и используя перебор по сетке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - roc-auc. В ответ запишите наилучшее среди искомых значение max_depth.

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

params = {'max_depth' : np.arange(2, 20, 1)}

gs = GridSearchCV(DecisionTreeClassifier(), params, cv=3, scoring='roc_auc')
gs.fit(X, y)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])},
             scoring='roc_auc')

In [ ]:
gs.best_params_

{'max_depth': 3}

In [ ]:
gs.best_score_

0.9938959367530796

In [ ]:
?DecisionTreeClassifier